In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class Polynomial_Regression:
    def __init__(self, degree=3, num=1000):
        self.degree = degree
        self.num = num
        self.scaler = Scaler()

    def _poly_features(self, X):
        base = X.copy()
        out = X.copy()

        for d in range(2, self.degree + 1):
            for col in base.columns:
                out[f"{col}^{d}"] = base[col] ** d
        return out

    def fit(self, X, y):
        X_poly = self._poly_features(X)

        self.scaler.scale(X_poly)
        X_poly = self.scaler.transform(X_poly)

        lr = Linear_Regression(self.num)
        self.intercept, self.coef = lr.fit(X_poly, y)

    def predict(self, X):
        X_poly = self._poly_features(X)
        X_poly = self.scaler.transform(X_poly)
        return self.intercept + np.dot(X_poly.values, self.coef)

class Linear_Regression:
    def __init__(self,numb=1000,learning_rate=0.01):
        self.numb = numb
        self.rate = learning_rate
    def fit(self,x_train,y_train):
        self.x_train = np.array(x_train)
        self.y_train = np.array(y_train)
        self.coffe = np.ones(self.x_train.shape[1])
        self.inter = 0
        self.v_inter = 0
        self.v_coffe = np.zeros(self.x_train.shape[1])
        beta = 0.9
        
        for i in range(self.numb):
            y_hat = np.dot(self.x_train,self.coffe) + self.inter
            intercept_derivative = (-2/(self.x_train.shape[0]))*(np.sum(self.y_train-y_hat))
            cofficenet_derivative = (-2/(self.x_train.shape[0]))*(np.dot((self.x_train.T),(self.y_train-y_hat)))

            self.v_coffe = beta * self.v_coffe + cofficenet_derivative
            self.v_inter = beta * self.v_inter + intercept_derivative

            
            self.inter = self.inter - self.rate*self.v_inter
            self.coffe = self.coffe - self.rate*self.v_coffe
        self.y_hat_pred = np.dot(self.x_train,self.coffe)+self.inter
        return self.inter, self.coffe


    def predict(self,x_test):
        x_test = np.array(x_test)
        self.predict_result = self.inter + np.dot(x_test,self.coffe)
        self.x_test = x_test
        return self.predict_result
    
    
    def r2_score(self,y_pre,y_test):
        
        y_mean = np.mean(y_test)

        ss_r = np.sum((y_test - y_pre)**2)
        ss_m = np.sum((y_test - y_mean)**2)
        
        self.r2 = 1-ss_r/ss_m

        return self.r2
        
    def adjusted_r2_score(self):
        row = self.x_train.shape[0]
        col = self.x_train.shape[1]
        self.adjusted = 1-((1-self.r2)*(row-1)/(row-1-col))
        return self.adjusted

class Scaler:
    def __init__(self):
        self.mean = None
        self.std = None
        
    def scale(self,dataset):
        
        self.mean = dataset.mean()
        self.std = dataset.std()
        
    
    def transform(self,dataset):
        new_dataset = (dataset-self.mean)/(self.std)
        return new_dataset
    
class Bagging_Linear:
    def __init__(self,percent=0.7,lr_count=5,epoch=1000,rate=0.01):
        self.epoch=epoch
        self.rate = rate
        self.count = lr_count
        self.percent = percent
    def fit(self,dataset):
        
        self.dataset = dataset

        bag_item = []

        for i in range(self.count):
            datasets = self.dataset.sample(int(dataset.shape[0]*(self.percent)),random_state=i,replace=True)
            bag_item.append(datasets)

        bag_item_target = []
        for i in range(self.count):
            target_item = bag_item[i]['target']
            bag_item[i]= bag_item[i].drop(columns=['target'])
            bag_item_target.append(target_item)

        self.lr_object = []
        for i in range(self.count):
            lrobject = Linear_Regression(numb=self.epoch,learning_rate=self.rate)
            lrobject.fit(bag_item[i],bag_item_target[i])
            self.lr_object.append(lrobject)
        print("""
        (\__/)
        (^_ ^)
        (>Training Completed
        """)
            
    def bag_predict(self,x_test):
        """
        l1_out = self.l1.predict(x_test)
        l2_out = self.l2.predict(x_test)
        l3_out = self.l3.predict(x_test)
        final_out = (l1_out+l2_out+l3_out)/3
        return final_out"""

        result_out = []
        for i in range(self.count):
            out = self.lr_object[i].predict(x_test)
            result_out.append(out)
        result_sum = 0
        for i in range(self.count):
            result_sum = result_sum+result_out[i]
        
        final_result = result_sum/(self.count)
        return final_result


class Train_test_split:
    def __init__(self,dataset,test_size=0.2,random=1,target='target'):
        self.target = target
        self.random_state=random
        self.test_size=test_size
        self.dataset = dataset
        
        self.row = self.dataset.shape[0]


    def split(self):
        
        np.random.seed(self.random_state)
        row_count = int(self.row*self.test_size)
        value = np.arange(0,self.row)
        row_no = np.random.choice(value,replace=False,size=row_count)
        

        test_dataset = self.dataset.iloc[row_no]
        x_test = test_dataset.drop(columns=[self.target])
        y_test = test_dataset[self.target]

        remaining_row=[]
        for i in range(self.row):
            if i not in row_no:
                remaining_row.append(i)
        train_dataset = self.dataset.iloc[remaining_row]
        x_train = train_dataset.drop(columns=[self.target])
        y_train = train_dataset[self.target]
        
        return x_train,x_test,y_train,y_test
def r2_score(y_pre,y_test):
        
    y_mean = np.mean(y_test)

    ss_r = np.sum((y_test - y_pre)**2)
    ss_m = np.sum((y_test - y_mean)**2)
        
    r2 = 1-ss_r/ss_m

    return r2
    

<>:129: SyntaxWarning: invalid escape sequence '\_'
<>:129: SyntaxWarning: invalid escape sequence '\_'
/var/folders/qx/m587h9hn4155cbn53_ttrdhc0000gn/T/ipykernel_2916/348759918.py:129: SyntaxWarning: invalid escape sequence '\_'
  (\__/)


In [22]:
data = pd.read_csv('poly_train_new_dataset.csv')
target = data.iloc[:,-1]

In [23]:
s = Scaler()
s.scale(data.iloc[:,:])
data = s.transform(data.iloc[:,:])


In [24]:
t = Train_test_split(data,target='y')
x_train,x_test,y_train,y_test = t.split()

In [25]:
poly = Polynomial_Regression(degree=4,num=1000)

In [26]:
poly.fit(x_train,y_train)

In [27]:
pre_value = poly.predict(x_test)

In [28]:
score = r2_score(pre_value,y_test)

In [29]:
score

np.float64(0.9910765390839511)

In [30]:
test_data = pd.read_csv("poly_test.csv")

y_test_pred = poly.predict(test_data)

y_test_pred = np.asarray(y_test_pred).ravel()

submit = pd.DataFrame({
    "id": np.arange(len(y_test_pred)),
    "y": y_test_pred
})

submit.to_csv("poly_submission.csv", index=False)